
<img src="../images/dask.png" width=200 alt="Dask Logo"></img>

# Kerchunk and Dask

## Overview
   
In this notebook we will cover:

1. How to parallelize the creation of `Kerchunk` reference files using the `Dask` library.
1. How to scale up the creation of large `Kerchunk` datasets using `Dask`. 

This notebook builds upon the [Kerchunk Basics](./kerchunk_basics.ipynb) and the [Multi-File Datasets with Kerchunk](../foundations/kerchunk_multi_file.ipynb) notebooks. A basic understanding of `Dask` will be helpful, but is not required. This notebook is not intended as a tutorial for using `Dask`, but will show how to use `Dask` to greatly speedup the the generation of `Kerchunk` reference files. 


## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Kerchunk Basics](../foundations/kerchunk_basics) | Required | Core |
| [Multiple Files and Kerchunk](../foundations/kerchunk_multi_file) | Required | Core |
| [Introduction to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Recommended | IO/Visualization |
| [Intro to Dask](https://tutorial.dask.org/00_overview.html) | Recommended | Parallel Processing |

- **Time to learn**: 45 minutes
---

## Dask and Parallel Processing

Dask is a `Python` library for parallel computing. It plays well with `Xarray`, but can be used in many ways across the `Python` ecosystem. This notebook is not intended to be a guide for how to use `Dask`, but just an example of how to use `Dask` to parallelize some `Kerchunk` functionality. 

In the previous notebook [Multiple Files and Kerchunk](../foundations/kerchunk_multi_file), we were looking at daily downscaled climate data over South-Eastern Alaska. In our function named `generate_json_reference`, we were looping over, one at a time, the input NetCDF4 files and using `Kerchunk's` `SingleHdf5ToZarr` method to create a `Kerchunk` index for each file.

With `Dask`, we can call `SingleHdf5ToZarr` in parallel, which allows us to create multiple `Kerchunk` reference files at the same time.

Further on in this notebook, we will show how using `Dask` can greatly speed-up the process of creating a virtual dataset using `Kerchunk`



## Setting up the `Dask` Client

In the code below, we are importing `Dask Disributed` and creating a `client`. This is the start of our parallel `Kerchunk` data processing. We are passing the argument `n_workers=8`. This will inform the `Dask` client on some of the resource limitations. 

Note: Depending on if you are running on a small machine such as a laptop or a larger compute hub, these resources could be tuned to improve performance. 

In [ ]:
import logging

from distributed import Client

client = Client(n_workers=8, silence_logs=logging.ERROR)
client

## Building off of our Previous Work
In the next section, we will re-use some of the code from [Multiple Files and Kerchunk](../foundations/kerchunk_multi_file) notebook. However, we will modify it slightly to make it compatable with `Dask`.

The following two cells should look the same as before.  As a reminder we are importing the required libraries, using `fsspec` to create a list of our input files and setting up some kwargs for `fsspec` to use. 

In [ ]:
import os

import dask
import fsspec
import ujson
import xarray as xr
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr

In [ ]:
# Initiate fsspec filesystems for reading and writing
fs_read = fsspec.filesystem("s3", anon=True, skip_instance_cache=True)
fs_write = fsspec.filesystem("")

# Retrieve list of available days in archive for the year 2060.
files_paths = fs_read.glob("s3://wrf-se-ak-ar5/ccsm/rcp85/daily/2060/*")

# Here we prepend the prefix 's3://', which points to AWS.
file_pattern = sorted(["s3://" + f for f in files_paths])

# Define kwargs for `fsspec`
so = dict(mode="rb", anon=True, default_fill_cache=False, default_cache_type="first")

output_dir = "./json_dir"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Subset the Data
To speed up our example, lets take a subset of the year of data. 

In [ ]:
file_pattern = file_pattern[0:40]

## Dask Specific Changes

Here is the section of code that will change. Instead of iterating through each input file and using `generate_json_reference` to create the `Kerchunk` reference files, we are iterating through our input file list and creating `Dask Delayed Objects`. It is not super important to understand this, but a `Dask Delayed Object` is lazy, meaning it is not computed eagerly. Once we have iterated through all our input files, we end up with a list of `Dask Delayed Objects`. 

When we are ready, we can call `dask.compute` on this list of delayed objects to create  `Kerchunk` reference files in parallel. 



In [ ]:
# Use Kerchunk's `SingleHdf5ToZarr` method to create a `Kerchunk` index from a NetCDF file.
def generate_json_reference(fil, output_dir: str):
    with fs_read.open(fil, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, fil, inline_threshold=300)
        fname = fil.split("/")[-1].strip(".nc")
        outf = f"{output_dir}/{fname}.json"
        with open(outf, "wb") as f:
            f.write(ujson.dumps(h5chunks.translate()).encode())
        return outf


# Generate Dask Delayed objects
tasks = [dask.delayed(generate_json_reference)(fil, output_dir) for fil in file_pattern]

## Dask Task Graph

Once you call `dask.compute` it can be hard to understand what is happening and how far along the process is at any time. Fortunately, `Dask` has a built in dashboard to help visualize your progress.

When you first initialized the `Dask` `client` earlier on, it should have returned some information including an address to the `dashboard`. For example: `http://127.0.0.1:8787/status`


By navigating to this address, you should a `Dask` dashboard that looks something like this.

<br/><br/>

<img src="../images/empty_dashboard.png" width="700"/> 
  
<br/><br/>

When you call `dask.compute(tasks)`, the dashboard should populate with a bunch of tasks. In the dashboard you can monitor your progress, see how resources are being used as well as well as countless other functionality. 

<br/><br/>

<img src="../images/dashboard.png"  width="700"/> 

<br/><br/>

## Start the Dask Processing

In [ ]:
# Start parallel computation of `SingleHDF5ToZarr`
dask.compute(tasks)

## Timing


| Serial Kerchunk      | Parallel Kerchunk (Dask) |
| -------------------- | ------------------------ |
| 225 seconds per file | 54 seconds per file      |



Running our `Dask` version on a subset of 40 files took about 36 minutes, or 54 second per file. In comparison, computing the kerchunk indicies one-by-one in [Multi-File Datasets with Kerchunk](../foundations/kerchunk_multi_file.ipynb) took about 3 minutes and 45 seconds for each file. 



Just by changing a few lines of code and using `Dask`, we got our code to run over **4x faster**. One other detail to note is that there is usually a bit of a delay as `Dask` builds its task graph before any of the tasks are started. All that to say, you may see even better performance when using `Dask` and `Kerchunk` on larger datasets.

Note: These timings may vary for you. There are many factors that may affect performance, such as:
- Geographical location of your compute and the source data
- Internet speed
- Compute resources, IO limits and # of workers given to `Dask`
- Location of where to write reference files to (cloud vs local)
- Type of reference files (`Parquet` vs `.JSON`)

This is meant to be an example of how `Dask` can be used to speed-up `Kerchunk` not a detailed benchmark in `Kerchunk/Dask` performance. 





## Next Steps

In this notebook we demonstrated how `Dask` can be used to parallelize the creation of `Kerchunk` reference files. In the following `Case Studies` section, we will walk though examples of using `Kerchunk` with `Dask` to create virtual cloud-optimized datasets. 

Additionally, if you wish to explore more of `Kerchunk's` `Dask` integration, you can try checking out `Kerchunk's` [auto_dask](https://fsspec.github.io/kerchunk/reference.html?highlight=auto%20dask#kerchunk.combine.auto_dask) method, which combines many of the `Dask` setup steps into a single convenient function. 

